In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import time
import statistics
import sklearn
%matplotlib inline
from datetime import date
from scipy import stats as stats
from sklearn import preprocessing
from sklearn import model_selection
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler

In [2]:
df_ADMISSIONS = pd.read_csv("/Users/alex/ADMISSIONS.csv")
col_list = ["SUBJECT_ID", "DOB", "GENDER", "EXPIRE_FLAG"]
df_PATIENTS = pd.read_csv("/Users/alex/PATIENTS.csv", usecols=col_list)

df_ADMISSIONS = pd.merge(df_ADMISSIONS, df_PATIENTS,on='SUBJECT_ID',how='outer',indicator=True)

##Finds the difference between admission and discharge time and puts it into a list in hours

date_mask = "%Y-%m-%d %H:%M:%S"
df_ADMISSIONS['ADMITTIME'] = pd.to_datetime(df_ADMISSIONS['ADMITTIME'], format= date_mask)
df_ADMISSIONS['DISCHTIME'] = pd.to_datetime(df_ADMISSIONS['DISCHTIME'], format= date_mask)
df_ADMISSIONS['DOB'] = pd.to_datetime(df_ADMISSIONS['DOB'], format= date_mask)
df_ADMISSIONS['LOS_hr'] = round(((df_ADMISSIONS['DISCHTIME'] - df_ADMISSIONS['ADMITTIME']).dt.total_seconds() / 3600))

##Finds the differcne between DOB and admition time and works out the differnce in years
age = []
for i in range(0,58976):
    datetime1 = pd.to_datetime(df_ADMISSIONS.DOB[i], format=date_mask)
    datetime2 = pd.to_datetime(df_ADMISSIONS.ADMITTIME[i], format=date_mask)
    days = datetime2 - datetime1
    years_old = int(round(days.days/(366)))
    age.append(years_old)
df_ADMISSIONS['AGE'] = age

##Replaces 300 year old patients with the 91 year old median value
for i in range(0, 58976):
    if df_ADMISSIONS.AGE[i] > 140:
        df_ADMISSIONS["AGE"].replace(df_ADMISSIONS.AGE[i], 91, inplace = True)

In [3]:
# Drops Patient Records from ADMISSIONS based on exclusion criteria in paper.
df_ADMISSIONS = df_ADMISSIONS.drop(df_ADMISSIONS[df_ADMISSIONS.EXPIRE_FLAG == 1].index)
df_ADMISSIONS = df_ADMISSIONS.drop_duplicates(subset='SUBJECT_ID', keep='first')
df_ADMISSIONS = df_ADMISSIONS.drop(df_ADMISSIONS[df_ADMISSIONS.HAS_CHARTEVENTS_DATA == 0].index)
df_ADMISSIONS = df_ADMISSIONS.drop(df_ADMISSIONS[df_ADMISSIONS.AGE < 15].index)
df_ADMISSIONS = df_ADMISSIONS.drop(df_ADMISSIONS[df_ADMISSIONS.LOS_hr < 0.1].index)

In [4]:
#Opens columns that appear in 'col_list' from CHARTEVENTS and OUTPUTEVENTS
col_list = ["HADM_ID", "ITEMID", 'CHARTTIME', 'VALUE']
df_OUTPUT = pd.read_csv("/Users/alex/OUTPUTEVENTS.csv", usecols=col_list, chunksize = 1000000)
df_EVENTS = pd.read_csv("/Users/alex/CHARTEVENTS.csv", usecols=col_list, chunksize = 1000000)

#Used to pair events with patient stays
df_HADM = df_ADMISSIONS[['HADM_ID']]

#Goes through chunks merging events that match HADM_ID of patients to df_HADM
def preprocessing(chunk, df, lst):
    df = pd.DataFrame(chunk)
    common = df_HADM.merge(df,on=['HADM_ID','HADM_ID'])
    df_HADM[(~df_HADM.HADM_ID.isin(common.HADM_ID))&(~df_HADM.HADM_ID.isin(common.HADM_ID))]
    common = pd.DataFrame(common)
    lst.append(common) 


output_lst = []
for chunk in df_OUTPUT:
    preprocessing(chunk, df_OUTPUT, output_lst)  
output = pd.concat(output_lst)


events_lst = []
for chunk in df_EVENTS:
    preprocessing(chunk, df_EVENTS, events_lst)  
events = pd.concat(events_lst)

/Users/alex/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
#This function searches CHARTEVENTS for the ITEMIDs given in each list and forms them into a df named by the vital 
#it contains
def sort_vitals(Vital_name,Vital, maxval, minval):
    boolean_series = events.ITEMID.isin(Vital)
    filtered_df = events[boolean_series].copy()
    filtered_df['VALUE'] = pd.to_numeric(filtered_df['VALUE'], errors='coerce')
    filtered_df.sort_values(by=['HADM_ID', 'CHARTTIME'])
    filtered_df.fillna(method='ffill', inplace=True)
    filtered_df['ITEM_NAME'] = Vital_name
    filtered_df = filtered_df.drop(filtered_df[filtered_df.VALUE > maxval].index)
    Vital = pd.DataFrame(filtered_df.drop(filtered_df[filtered_df.VALUE < minval].index))
    return Vital


systolicBP = [227243, 224167, 220179, 225309, 220050, 442, 455, 6701, 51]
SystolicBP = sort_vitals('SystolicBP', systolicBP, 450, 0)

Potassium =[227464, 829, 1535]
Potassium = sort_vitals('Potassium', Potassium, 1000, 0)

white_blood_cell = [220546, 861, 1127, 1542]
White_Blood_Cell = sort_vitals('WBC', white_blood_cell, 1000, 0)

sodium = [220645, 226534, 837, 1536]
Sodium = sort_vitals('Sodium', sodium, 10000, 0)

bicarbonate = [227443]
Bicarbonate = sort_vitals('Bicarbonate', bicarbonate, 1000, 0)

glasgow_coma = [198]
Glasgow_Coma = sort_vitals('Glasgow_Coma', glasgow_coma, 1000, 0)

birillium = [225690, 848, 1538, 225651, 803]
Birillium = sort_vitals('Birillium', birillium, 10000, 0)

Blood_Urea_Nitrogen = [225624, 781, 1162]
BloodUreaNitrogen = sort_vitals('BloodUreaNitrogen', Blood_Urea_Nitrogen, 50000, 0)

mech_vent = [223835, 3420, 3422, 190]
Mech_Vent = sort_vitals('Mech_Vent', mech_vent, 50000, 0)

Respiratory_Rate = [220210, 224690, 224689, 618, 614, 651, 615]
Respiratory_Rate = sort_vitals('Respiratory_Rate', Respiratory_Rate, 650, 0)

creatine = [220615, 791, 1525]
Creatine = sort_vitals('Creatine', creatine, 1000, 0)

diastolicBP = [224643, 220180, 220051,225310, 8441, 8555, 8368, 8440]
diastolicBP = sort_vitals('diastolicBP', diastolicBP, 1000, 0)

Oxygen_Saturation = [220277, 646, 834]
Oxygen_Saturation = sort_vitals('Oxygen_Saturation', Oxygen_Saturation, 100, 0.1)

hematocrit = [220545, 813]
Hematocrit = sort_vitals('Hematocrit',hematocrit, 1000, 0)

calcium = [225625, 786, 1522]
Calcium = sort_vitals('Calcium', calcium, 50000, 0)

magnesium = [220635, 821, 1532]
Magnesium = sort_vitals('Magnesium', magnesium, 1000, 0)

Albumin = [227456, 772, 1521]
Albumin = sort_vitals('Albumin', Albumin, 50000, 0)

pH = [220274, 860, 780, 223830, 1126]
pH = sort_vitals('pH', pH, 10, 4)

mean_BP = [220181, 224322, 225312, 220052, 224, 456, 6702, 52]
Mean_BP = sort_vitals('Mean_BP', mean_BP, 500, 0)

lactic_acid = [225668, 818, 1531]
Lactic_Acid = sort_vitals('Lactic_Acid', lactic_acid, 200, 0)

platelets = [227457, 828]
Platelets = sort_vitals('Platelets', platelets, 10000, 0)

red_blood = [833]
Red_Blood = sort_vitals('Red_Blood', red_blood, 10000000, 0)

white_blood_cell = [220546, 861, 1127, 1542]
White_Blood_Cell = sort_vitals('White_blood_cell',white_blood_cell, 1000, 0)

#These vitals required special manipulation so have their own code.

weight = [224639, 226512, 226531, 763]
boolean_series = events.ITEMID.isin(weight)
filtered_df = events[boolean_series].copy()
filtered_df['VALUE'] = pd.to_numeric(filtered_df['VALUE'], errors='coerce')
filtered_df.sort_values(by=['HADM_ID', 'CHARTTIME'])
filtered_df.fillna(method='ffill', inplace=True)
filtered_df['VALUE'] = np.where(filtered_df['ITEMID'] == 226531, filtered_df['VALUE'] // 2.2, filtered_df['VALUE'])
filtered_df = filtered_df.drop(filtered_df[filtered_df.VALUE > 1000].index)
filtered_df['ITEM_NAME'] = 'Weight'
Weight = filtered_df.drop(filtered_df[filtered_df.VALUE < 0.5].index)

HeartRate = [220045, 211]
boolean_series = events.ITEMID.isin(HeartRate)
filtered_df = events[boolean_series].copy()
filtered_df['VALUE'] = pd.to_numeric(filtered_df['VALUE'], errors='coerce')
filtered_df.sort_values(by=['HADM_ID', 'CHARTTIME'])
filtered_df.fillna(method='ffill', inplace=True)
filtered_df.VALUE = pd.to_numeric(filtered_df.VALUE)
filtered_df['ITEM_NAME'] = 'HeartRate'
HeartRate = pd.DataFrame(filtered_df.drop(filtered_df[filtered_df.VALUE > 480].index))

temperature = [223761, 223762, 676, 677, 678, 679]
boolean_series = events.ITEMID.isin(temperature)
filtered_df = events[boolean_series].copy()
filtered_df['VALUE'] = pd.to_numeric(filtered_df['VALUE'], errors='coerce')
filtered_df.sort_values(by=['HADM_ID', 'CHARTTIME'])
filtered_df.fillna(method='ffill', inplace=True)
filtered_df['VALUE'] = np.where(filtered_df['ITEMID'] == 223761, ((filtered_df['VALUE'] - 32) * 5/9), filtered_df['VALUE'])
filtered_df = filtered_df.drop(filtered_df[filtered_df.VALUE > 47].index)
filtered_df['ITEM_NAME'] = "Temperature"
Temperature = pd.DataFrame(filtered_df.drop(filtered_df[filtered_df.VALUE < 10].index))

urine = [40055, 43175, 40069, 40094, 40715, 40473, 40085, 40057, 40056, 40405, 40428, 40086,
         40096, 40651, 226559, 226560, 226561, 226584, 226563, 226564, 226565, 226567, 226557, 
         226558, 227488, 227489]
boolean_series = output.ITEMID.isin(urine)
filtered_df = output[boolean_series].copy()
filtered_df['VALUE'] = pd.to_numeric(filtered_df['VALUE'], errors='coerce')
filtered_df.sort_values(by=['HADM_ID', 'CHARTTIME'])
filtered_df.fillna(method='ffill', inplace=True)
filtered_df['ITEM_NAME'] = "Urine"
Urine = filtered_df.drop(filtered_df[filtered_df.VALUE > 50000].index)
Urine = pd.DataFrame(filtered_df.drop(filtered_df[filtered_df.VALUE < 0].index))

#Urine was only available from OUTPUT not EVENTS so this searches OUTPUT instead of EVENTS
col_list = ["ITEMID", 'VALUEUOM']
df_VALUEUOM = pd.read_csv("/Users/alex/OUTPUTEVENTS.csv", usecols=col_list, nrows = 10000000)
df_VALUEUOM = df_VALUEUOM.drop_duplicates(subset='ITEMID', keep='first')
show = df_VALUEUOM.loc[df_VALUEUOM['ITEMID'].isin(urine)]

In [6]:
lst = [Mech_Vent, Urine, BloodUreaNitrogen, Birillium, Glasgow_Coma, Bicarbonate, Sodium, White_Blood_Cell,
      Temperature, Potassium, SystolicBP, HeartRate, Red_Blood, Platelets, Lactic_Acid, Mean_BP, Weight, pH, 
     Albumin, Magnesium, Calcium, Hematocrit, Oxygen_Saturation, diastolicBP, Creatine, Respiratory_Rate]
df = pd.concat(lst, sort = False)

In [7]:
#Fetches relevant columns from DIAGNOSES list
diagnose_list = ['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE']
df_DIAGNOSES = pd.read_csv("/Users/alex/DIAGNOSES_ICD.csv", usecols = diagnose_list)
df_HADM = df_HADM.reset_index(drop=True)

#Keeps diagnoses of patients who passed the elimination criteria 
common = df_HADM.merge(df_DIAGNOSES,on=['HADM_ID','HADM_ID'])
df_HADM[(~df_HADM.HADM_ID.isin(common.HADM_ID))&(~df_HADM.HADM_ID.isin(common.HADM_ID))]
df_DIAGNOSES = pd.DataFrame(common)

#Puts ICD9 codes into a list
ICD_9 = df_DIAGNOSES['ICD9_CODE'].tolist()
E = []

#Tranforms them into the first 3 characters if a number of beginning with V and first 4 if beginning with E
for i in range(0, len(ICD_9)):
    if ICD_9[i][0] == 'E':
        E.append(ICD_9[i][0:4])
    elif ICD_9[i][0] == 'V':
        E.append(ICD_9[i][0:3])
    else:
        E.append(ICD_9[i][0:3])
    

ICD_9 = E

#creates new dataframe to hold codes
df_ICD9CODES = df_DIAGNOSES[['ICD9_CODE', 'HADM_ID']]
df_ICD9CODES['ICD_CAT'] = ICD_9


#Dicitionary with the 18 categories codes will be sorted into
ICDDict = {
1 : 'infectious and parasitic',
2 : 'neoplasms',
3 : 'endocrine, metabolic and immunity',
4 : 'blood and blood-forming organs',
5 : 'mental',
6 : 'nervous',
7 : 'circulatory',
8 : 'respiratory',
9 : 'digestive',
10 : 'genitourinary',
11 : 'pregnancy, childbirth, and puerperium',
12 : 'skin and subcutaneous tissue',
13 : 'musculoskeletal',
14 : 'congenital',
15 : 'perinatal period',
16 : 'ill-defined',
17 : 'injury and poisoning',
18 : 'external causes of injury and supplemental classification'
}

NAME = []
CAT = []
#Goes through all ICD9 codes and appends the dictionary value for the ICD9 codes to list 'CAT'
for i in range(0, 236104):
    if ICD_9[i][0:1] == 'E' or ICD_9[i][0:1] == 'V':
        CAT.append(18)
    elif int(ICD_9[i]) <= 139:
        CAT.append(1)
    elif int(ICD_9[i]) > 139 and int(ICD_9[i]) <= 239:
        CAT.append(2)
    elif int(ICD_9[i]) > 239 and int(ICD_9[i]) <= 279:
        CAT.append(3)
    elif int(ICD_9[i]) > 279 and int(ICD_9[i]) <= 289:
        CAT.append(4)
    elif int(ICD_9[i]) > 289 and int(ICD_9[i]) <= 319:
        CAT.append(5)
    elif int(ICD_9[i]) > 319 and int(ICD_9[i]) <= 389:
        CAT.append(6)
    elif int(ICD_9[i]) > 389 and int(ICD_9[i]) <= 459:
        CAT.append(7)
    elif int(ICD_9[i]) > 460 and int(ICD_9[i]) <= 519:
        CAT.append(8)
    elif int(ICD_9[i]) > 519 and int(ICD_9[i]) <= 579:
        CAT.append(9)
    elif int(ICD_9[i]) > 579 and int(ICD_9[i]) <= 629:
        CAT.append(10)
    elif int(ICD_9[i]) > 629 and int(ICD_9[i]) <= 679:
        CAT.append(11)
    elif int(ICD_9[i]) > 679 and int(ICD_9[i]) <= 709:
        CAT.append(12)
    elif int(ICD_9[i]) > 709 and int(ICD_9[i]) <= 739:
        CAT.append(13)
    elif int(ICD_9[i]) > 739 and int(ICD_9[i]) <= 759:
        CAT.append(14)
    elif int(ICD_9[i]) > 759 and int(ICD_9[i]) <= 779:
        CAT.append(15)
    elif int(ICD_9[i]) > 779 and int(ICD_9[i]) <= 799:
        CAT.append(16)
    elif int(ICD_9[i]) > 799 and int(ICD_9[i]) <= 999:
        CAT.append(17)

#Creates new column for sorted codes and replaces the existing name with the name matched in the dictionary
df_ICD9CODES['CATEGORY_ICD9'] = CAT
df_ICD9CODES['ICD9_NAME'] = df_ICD9CODES['CATEGORY_ICD9'].replace(ICDDict)

#Groups 'df_ICD9CODES' by HADM_ID and resets index
ICD9HADMSort = df_ICD9CODES.groupby('HADM_ID')['ICD9_NAME'].apply(list).reset_index()

In [8]:
#Creates dummy variables for ICD9 Codes if you want to stakc, add .stack() after apply
ICD9_Dummy = pd.get_dummies(ICD9HADMSort['ICD9_NAME'].apply(pd.Series).stack(), drop_first=False).sum(level=0)

#Pairs them back with the inital data frame
ICD9_Dummy = ICD9_Dummy.join(ICD9HADMSort['HADM_ID'], how="outer")
ICD9_Dummy = ICD9_Dummy.replace([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 1)

In [9]:
#These dictionaries reallocate different entries to the final groups which are to be entered.

marriage_dictionary = {
   "SINGLE" : "SINGLE",
   "MARRIED" : "MARRIED",
   "DIVORCED" : "MARRIAGE ENDED",
   "WIDOWED" : "MARRIAGE ENDED",
   "SEPARATED" : "MARRIAGE ENDED",
   "UNKNOWN (DEFAULT)" : "UNKNOWN",
   "LIFE PARTNER" : "MARRIED"
                      }

df_ADMISSIONS.MARITAL_STATUS.replace(marriage_dictionary, inplace=True)


ethnicity_dictionary = {
    'WHITE' : 'WHITE',
    'UNKNOWN/NOT SPECIFIED' : 'UNKNOWN',
    'MULTI RACE ETHNICITY' : 'UNKNOWN',
    'BLACK/AFRICAN AMERICAN' : 'BLACK',
    'HISPANIC OR LATINO' : 'HISPANIC',
    'PATIENT DECLINED TO ANSWER' : 'UNKNOWN',
    'ASIAN': 'ASIAN',
    'OTHER' : 'OTHER',
    'HISPANIC/LATINO - GUATEMALAN' : 'HISPANIC',
    'ASIAN - VIETNAMESE' : 'ASIAN',
    'AMERICAN INDIAN/ALASKA NATIVE' : 'OTHER',
    'WHITE - RUSSIAN' : 'WHITE',
    'HISPANIC/LATINO - PUERTO RICAN' : 'HISPANIC',
    'ASIAN - ASIAN INDIAN' : 'ASIAN',
    'HISPANIC/LATINO - SALVADORAN' : 'HISPANIC',
    'HISPANIC/LATINO - DOMINICAN' : 'HISPANIC',
    'UNABLE TO OBTAIN' : 'UNKNOWN',
    'ASIAN - CHINESE' : 'ASIAN',
    'BLACK/CAPE VERDEAN' : 'BLACK',
    'WHITE - OTHER EUROPEAN' : 'WHITE',
    'PORTUGUESE' : 'WHITE',
    'BLACK/HAITIAN' : 'BLACK',
    'SOUTH AMERICAN' : 'HISPANIC',
    'CARIBBEAN ISLAND' : 'BLACK',
    'WHITE - EASTERN EUROPEAN' : 'WHITE',
    'ASIAN - FILIPINO' : 'ASIAN',
    'ASIAN - CAMBODIAN' : 'ASIAN',
    'HISPANIC/LATINO - CENTRAL AMERICAN (OTHER)' : 'HISPANIC',
    'WHITE - BRAZILIAN' : 'WHITE',
    'HISPANIC/LATINO - COLOMBIAN' : 'HISPANIC',
    'ASIAN - JAPANESE' : 'ASIAN',
    'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER' : 'OTHER',
    'BLACK/AFRICAN' : 'BLACK',
    'ASIAN - THAI' : 'ASIAN',
    'HISPANIC/LATINO - HONDURAN' : 'HISPANIC',
    'MIDDLE EASTERN' : 'OTHER',
    'ASIAN - OTHER' : 'ASIAN',
    'HISPANIC/LATINO - CUBAN' : 'HISPANIC',
    'HISPANIC/LATINO - MEXICAN' : 'HISPANIC',
    'ASIAN - KOREAN' : 'ASIAN',
    'AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE' : 'OTHER'
                       }
df_ADMISSIONS.ETHNICITY.replace(ethnicity_dictionary, inplace=True)


religion_dictionary = {
    'UNOBTAINABLE' : 'UNKNOWN',
    'CATHOLIC' : 'CATHOLIC',
    'PROTESTANT QUAKER' : 'PROTESTANT',
    'NOT SPECIFIED' : 'UNKNOWN',
    'JEWISH' : 'JEWISH',
    'BUDDHIST' : 'NON-CHRISTIAN OTHER',
    'OTHER' : 'UNKNOWN',
    "JEHOVAH'S WITNESS" : 'CHRISTIAN OTHER',
    'GREEK ORTHODOX' : 'CHRISTIAN OTHER',
    'EPISCOPALIAN' : 'CHRISTIAN OTHER',
    'HINDU' : 'NON-CHRISTIAN OTHER',
    'CHRISTIAN SCIENTIST' : 'CHRISTIAN OTHER',
    'HEBREW' : 'JEWISH',
    'METHODIST' : 'CHRISTIAN OTHER',
    'UNITARIAN-UNIVERSALIST' : 'CHRISTIAN OTHER',
    'BAPTIST' : 'CHRISTIAN OTHER',
    '7TH DAY ADVENTIST' : 'CHRISTIAN OTHER',
    'MUSLIM' : 'NON-CHRISTIAN OTHER',
    'ROMANIAN EAST. ORTH' : 'CHRISTIAN OTHER',
    'LUTHERAN' : 'CHRISTIAN OTHER'
}
df_ADMISSIONS.RELIGION.replace(religion_dictionary, inplace=True)


admission_location_dictionary = {
    'EMERGENCY ROOM ADMIT' : 'EMERGENCY ROOM ADMIT',
    'PHYS REFERRAL/NORMAL DELI' : 'PHYS REFERRAL/NORMAL DELI',
    'TRANSFER FROM HOSP/EXTRAM' : 'TRANSFER FROM HOSP/EXTRAM',
    'CLINIC REFERRAL/PREMATURE' : 'CLINIC REFERRAL/PREMATURE', 
    'TRANSFER FROM SKILLED NUR' : 'TRANSFER FROM SKILLED NUR',
    'HMO REFERRAL/SICK' : 'HMO REFERRAL/SICK',
    '** INFO NOT AVAILABLE **' : 'UNKNOWN',
    'TRANSFER FROM OTHER HEALT' : 'TRANSFER FROM OTHER HEALT',
    'TRSF WITHIN THIS FACILITY' : 'TRSF WITHIN THIS FACILITY'
}
df_ADMISSIONS.ADMISSION_LOCATION.replace(admission_location_dictionary, inplace=True)


admission_type_dictionary = {
    'EMERGENCY' : 'EMERGENCY',
    'ELECTIVE' : 'ELECTIVE',
    'NEWBORN' : 'NEWBORN',
    'URGENT' : 'URGENT'
}
df_ADMISSIONS.ADMISSION_TYPE.replace(admission_type_dictionary, inplace=True)

insurance_dictionary = {
    'Private' : 'PRIVATE', 
    'Medicare' : 'GOVERNMENT', 
    'Medicaid' : 'GOVERNMENT', 
    'Self Pay' : 'SELF-PAY', 
    'Government' : 'GOVERNMENT'
}
df_ADMISSIONS.INSURANCE.replace(insurance_dictionary, inplace=True)



In [10]:
#Merges SAPSTime with ICD9_Dummy removing instances where HADM_ID does not match to anything
common = df.merge(ICD9_Dummy,on=['HADM_ID','HADM_ID'])
df[(~df.HADM_ID.isin(common.HADM_ID))&(~df.HADM_ID.isin(common.HADM_ID))]
df = pd.DataFrame(common)

df_Demo = df_ADMISSIONS[['AGE', 'HADM_ID', 'LOS_hr', 'ADMITTIME', 'DISCHTIME', 'ADMISSION_TYPE',
                        'ADMISSION_LOCATION', 'INSURANCE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY']]

In [11]:
#Remove hot one encodes categorical variables
admission_type = pd.get_dummies(df_Demo['ADMISSION_TYPE'], drop_first=False).sum(level = 0)
admission_location = pd.get_dummies(df_Demo['ADMISSION_LOCATION'], drop_first=False).sum(level = 0)
insurance = pd.get_dummies(df_Demo['INSURANCE'], drop_first=False).sum(level = 0)
religion = pd.get_dummies(df_Demo['RELIGION'], drop_first=False).sum(level = 0)
martial_status = pd.get_dummies(df_Demo['MARITAL_STATUS'], drop_first=False).sum(level = 0)
ethnicity = pd.get_dummies(df_Demo['ETHNICITY'], drop_first=False).sum(level = 0)

In [12]:
#Creates an array joining dummies together
df_dummy = []
df_dummy = admission_type.join(df_Demo['HADM_ID'], how = "outer")
df_dummy = admission_location.join(df_dummy, how = "outer")
df_dummy = insurance.join(df_dummy, how = "outer")
df_dummy = religion.join(df_dummy, how = "outer", lsuffix='religion')
df_dummy = martial_status.join(df_dummy, how = "outer", lsuffix='marriage')
df_dummy = ethnicity.join(df_dummy, how = "outer", lsuffix='ethnicity')

In [13]:
common = df_Demo.merge(df_dummy,on=['HADM_ID','HADM_ID'])
df_Demo[(~df_Demo.HADM_ID.isin(common.HADM_ID))&(~df_Demo.HADM_ID.isin(common.HADM_ID))]
df_Demo = pd.DataFrame(common)

In [14]:
#Drops names of categories leaving only hot one encodings
df_Demo = df_Demo.drop(['ADMISSION_TYPE'], axis = 1)
df_Demo = df_Demo.drop(['ADMISSION_LOCATION'], axis = 1)
df_Demo = df_Demo.drop(['INSURANCE'], axis = 1)
df_Demo = df_Demo.drop(['RELIGION'], axis = 1)
df_Demo = df_Demo.drop(['MARITAL_STATUS'], axis = 1)
df_Demo = df_Demo.drop(['ETHNICITY'], axis = 1)

In [15]:
#Creates bins for ages and hot one encodes them.
pd.cut(df_Demo.AGE, 10)
bins = [14, 40, 59, 69, 74, 79, 99]
labels =['1','2','3','4', '5', '6']
df_Demo['AGE BIN'] = pd.cut(df_Demo['AGE'], bins,labels=labels)
df_Demo = df_Demo.drop(['AGE'], axis = 1)

age_item = pd.get_dummies(df_Demo['AGE BIN'], prefix='Age Bin', drop_first=True)
df_Demo = df_Demo.join(age_item)

In [18]:
common = df.merge(df_Demo,on=['HADM_ID','HADM_ID'])
df[(~df.HADM_ID.isin(common.HADM_ID))&(~df.HADM_ID.isin(common.HADM_ID))]
df = pd.DataFrame(common)

In [21]:
#Works out hours left, hours in and shifts left and rounds timestamps to nearest hour
df['ADMITTIME'] = pd.to_datetime(df['ADMITTIME'], format= date_mask)
df['CHARTTIME'] = pd.to_datetime(df['CHARTTIME'], format= date_mask)
df['DISCHTIME'] = pd.to_datetime(df['DISCHTIME'], format= date_mask)
df['HOURS_IN'] = round(((df['CHARTTIME'] - df['ADMITTIME']).dt.total_seconds() / 3600))
df['HOURS_LEFT'] = round(((df['DISCHTIME'] - df['CHARTTIME']).dt.total_seconds() / 3600))
df['SHIFTS_LEFT'] = round((df['HOURS_LEFT'] / 8))
df.sort_values(by='CHARTTIME')
df['CHARTTIME'] = pd.to_datetime(df['CHARTTIME']) 
df['CHARTTIME'] = df['CHARTTIME'].dt.round('H')

In [23]:
#Sorts dataframe by time and drops duplicates keeping the first 
df_time = df.sort_values(by=['CHARTTIME'], ascending = True)
df_time = df_time.drop_duplicates(subset=['CHARTTIME', 'HADM_ID', 'ITEM_NAME'], keep='first')
df_time = df_time.drop(df_time[df_time.LOS_hr < 4].index)

In [26]:
#Saves df to file '50varraw'
df_time.to_csv('50varraw.csv')